# **Writing Postgres Queries**
In order to write the API, I'll need a couple of different Postgres queries. I'll test them out throughout this notebook! 

# Setup
The cells below will set up the rest of the notebook.

I'll start by configuring the kernel: 

In [1]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

d:\data\programming\neural-needledrop\api


Now I'll import some necessary modules:

In [2]:
# General import statements
import pandas as pd
from pandas_gbq import read_gbq
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker, declarative_base
from tqdm import tqdm
from pathlib import Path
from google.cloud import storage
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np

# Importing custom modules 
from utils.openai import embed_text, embed_text_list
from utils.settings import (
    GBQ_PROJECT_ID,
    GBQ_DATASET_ID,
    POSTGRES_USER,
    POSTGRES_PASSWORD,
    POSTGRES_HOST,
    POSTGRES_PORT,
    POSTGRES_DB,
    LOG_TO_CONSOLE,
)
from utils.logging import get_logger
from utils.postgres import query_postgres, upload_to_table
from utils.postgres_queries import most_similar_embeddings

# Set up a logger for this notebook
logger = get_logger("postgres_notebook", log_to_console=LOG_TO_CONSOLE)

Finally, I'll set up some Postgres connectors: 

In [3]:
# Create the connection string to the database
postgres_connection_string = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

# Create the connection engine
engine = create_engine(postgres_connection_string)
metadata = MetaData()
session = sessionmaker(bind=engine)()
Base = declarative_base()

# **Query Experimentation**
Below, I've collected some of my experiments with writing the API queries.

### Searching for Similar Embeddings
The crux of this project: embedding arbitrary text, and then finding the most similar embeddings to that text.

In [4]:
# First: define the query, and then embed it
query_text = "Shredding guitar, heavy drums, and fast-paced vocals"
query_embedding = embed_text(query_text)

# Now, get the most similar embeddings
most_similar_embeddings(query_embedding, engine, n=5)

,id,url,embedding_type,start_segment,end_segment,segment_length,embedding,cos_sim
0,uCX9A3xROQo_24_28,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,24,28,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",1.0
1,uCX9A3xROQo_0_4,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,0,4,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",1.0
2,uCX9A3xROQo_4_8,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,4,8,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",1.0
3,uCX9A3xROQo_20_24,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,20,24,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",1.0
4,uCX9A3xROQo_28_32,https://www.youtube.com/watch?v=uCX9A3xROQo,segment_chunk,28,32,4,"[0.0030553022,0.03946543,-0.0013984011,0.01894...",1.0


### Retrieving Video Metadata
In order to display some information about a video, I'll need a general method to search for a bunch of their data. Should probably allow for the fetching of various videos' data, too. 

### Retrieving a Video's Transcript
Another method will be retrieving a video's entire transcript!

In [10]:
# Parameters
video_id = "uCX9A3xROQo"
video_url = f"https://www.youtube.com/watch?v={video_id}"

# Define the SQL query
sql_query = f"""
    SELECT 
        transcription.*
    FROM transcriptions  transcription
    WHERE
        url = '{video_url}'
    ORDER BY 
        segment_id ASC
"""

# Execute the query
df = query_postgres(sql_query, engine)

def retrieve_video_transcript(video_id, engine, logger=None):
    """"""